# Phase 2: Model Compression Pipeline

**Project:** Cogumi-LLM  
**Phase:** 2 - Compression (11GB → 480MB)  
**Input:** English-trained LLAMA-3.1-8B from Phase 1  
**Output:** 480MB compressed model  
**Duration:** 8-10 hours  
**GPU Required:** A100 40GB (or 80GB for faster processing)  

---

## Compression Pipeline

```
Phase 1 Output: 11GB English-specialized model
    ↓
Phase 2A: Neural Magic Pruning (65% removal)
    → 11GB → 3.85GB (5-6 hours)
    ↓
Phase 2B: AWQ Quantization (4-bit)
    → 3.85GB → 1.0GB (2-3 hours)
    ↓
Phase 2C: GGUF Export + Compression
    → 1.0GB → 480MB (1 hour)
    ↓
Final: 480MB English-specialized model
```

**Expected Quality:** 87-89% GPT-4  
**Total Time:** 8-10 hours  
**Cost:** ~$15-20 on Colab Pro+

---

## Setup Instructions

1. **Select Runtime**: Runtime → Change runtime type → A100 GPU
2. **Connect to GPU**: Click Connect in top-right
3. **Upload Phase 1 model** or sync from HuggingFace
4. **Run cells sequentially**
5. **Download compressed model**

⚠️ **Important**: This will take 8-10 hours. You can pause between phases if needed.

## 1. Environment Setup

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Verify we have A100
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda if hasattr(torch, 'version') and hasattr(torch.version, 'cuda') else 'N/A'}")  # type: ignore
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    
    # Verify it's A100
    gpu_name = torch.cuda.get_device_name(0)
    if 'A100' not in gpu_name:
        print("\n⚠️ WARNING: You need A100 GPU for compression!")
        print("Go to Runtime → Change runtime type → Select A100")
    else:
        print("\n✅ A100 GPU detected! Ready for compression.")
else:
    print("\n⚠️ WARNING: CUDA not available! Make sure you're using GPU runtime.")

## 2. Install Compression Tools

Installing:
- **SparseML**: Neural Magic's structured pruning
- **AutoAWQ**: 4-bit activation-aware quantization  
- **llama.cpp**: GGUF export and final compression

⏱️ **Estimated time: 5-7 minutes**

In [ ]:
print("=" * 60)
print("📦 INSTALLING COMPRESSION TOOLS")
print("=" * 60)

# Install SparseML for structured pruning
print("\n1. Installing Neural Magic SparseML...")
%pip install -q sparseml[transformers]

# Install AutoAWQ for 4-bit quantization
print("\n2. Installing AutoAWQ...")
%pip install -q autoawq

# Install llama.cpp tools
print("\n3. Setting up llama.cpp...")
!git clone https://github.com/ggerganov/llama.cpp /content/llama.cpp
!cd /content/llama.cpp && make -j 4

# Install additional utilities
print("\n4. Installing utilities...")
%pip install -q zstandard onnx onnxruntime

print("\n" + "=" * 60)
print("✅ All compression tools installed!")
print("=" * 60)

In [ ]:
# Verify installations
print("🔍 Verifying installations...\n")

try:
    import sparseml  # type: ignore
    print(f"✅ SparseML {sparseml.__version__}")
except:
    print("❌ SparseML not installed")

try:
    import awq  # type: ignore
    print(f"✅ AutoAWQ installed")
except:
    print("❌ AutoAWQ not installed")

try:
    import zstandard
    print(f"✅ Zstandard {zstandard.__version__}")
except:
    print("❌ Zstandard not installed")

import os
if os.path.exists('/content/llama.cpp/main'):
    print(f"✅ llama.cpp built successfully")
else:
    print("❌ llama.cpp not built")

print("\n✅ All tools ready for compression!")

## 3. Load Phase 1 Model

**Options:**
- **Option A**: Upload from HuggingFace Hub (recommended)
- **Option B**: Upload from Google Drive
- **Option C**: Upload from local machine

### Option A: Load from HuggingFace Hub (Recommended)

In [ ]:
# If you uploaded Phase 1 model to HuggingFace
from huggingface_hub import login

# Paste your HuggingFace token
HF_TOKEN = "YOUR_HF_TOKEN_HERE"

login(token=HF_TOKEN)
print("✅ HuggingFace authentication successful!")

In [ ]:
# Download Phase 1 model from HuggingFace
from transformers import AutoModelForCausalLM, AutoTokenizer

# Replace with your model ID
MODEL_ID = "YOUR_USERNAME/cogumi-llm-phase1a"  # Or wherever you uploaded it

print(f"📥 Downloading model from {MODEL_ID}...")
print("⏱️  This will take 10-15 minutes...\n")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

print("\n✅ Model loaded successfully!")
print(f"Model size: {sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters")

### Option B: Load from Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive  # type: ignore
drive.mount('/content/drive')

# Set path to your model in Drive
MODEL_PATH = "/content/drive/MyDrive/models/llama-3.1-8b-phase1a-merged"

print(f"📥 Loading model from {MODEL_PATH}...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

print("\n✅ Model loaded successfully!")

## 4. Prepare Calibration Dataset

We need ~512 samples from your training data for calibration during pruning and quantization.

In [ ]:
# Clone repository to get calibration data
import os

if os.path.exists('Cogumi-LLM'):
    print("📂 Repository already exists")
    %cd Cogumi-LLM
else:
    print("📥 Cloning repository...")
    !git clone https://github.com/dkeviv/Cogumi-LLM.git
    %cd Cogumi-LLM

In [ ]:
# Upload calibration dataset (or download from Drive)
# We only need a small subset for calibration

import json

def load_calibration_data(dataset_path, num_samples=512):
    """Load calibration samples."""
    samples = []
    
    print(f"📊 Loading {num_samples} calibration samples...")
    
    with open(dataset_path, 'r') as f:
        for i, line in enumerate(f):
            if i >= num_samples:
                break
            data = json.loads(line)
            text = f"{data['instruction']}\n\n{data['response']}"
            samples.append(text)
    
    print(f"✅ Loaded {len(samples)} calibration samples")
    return samples

# Load calibration data
# You'll need to upload data/phase1/public_500k_filtered.jsonl or a subset
calibration_data = load_calibration_data(
    'data/phase1/public_500k_filtered.jsonl',
    num_samples=512
)

## 5. Phase 2A: Neural Magic Structured Pruning

**Goal:** Remove 65% of neurons using structured 2:4 sparsity  
**Input:** 11GB model  
**Output:** 3.85GB pruned model  
**Time:** 5-6 hours  

---

### How it works:
- Applies 2:4 structured sparsity (2 zeros per 4 weights)
- CPU-optimized patterns (great for M4 Pro, Apple Silicon)
- Removes weakest neurons (non-English pathways!)
- Uses calibration data to preserve important weights

In [ ]:
from sparseml.transformers import oneshot  # type: ignore

print("=" * 60)
print("🔪 PHASE 2A: NEURAL MAGIC PRUNING")
print("=" * 60)
print("Target: 11GB → 3.85GB (65% sparsity)")
print("Method: Structured pruning with SparseML")
print("Duration: ~2-3 hours\n")

# Pruning recipe for 65% sparsity
pruning_config = {
    "sparsity": 0.65,
    "pruning_method": "magnitude",
    "targets": ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
}

print("📋 Pruning configuration:")
for k, v in pruning_config.items():
    print(f"  {k}: {v}")

# Apply one-shot pruning
pruned_model = oneshot(
    model=model,
    dataset=calibration_dataset,
    recipe=pruning_config,
    output_dir="models/llama-phase2a-pruned"
)

print("\n✅ Pruning complete!")
print(f"📊 Model sparsity: 65%")
print(f"💾 Saved to: models/llama-phase2a-pruned")

# Save tokenizer
tokenizer.save_pretrained("models/llama-phase2a-pruned")

print("\n📏 Size comparison:")
print(f"  Original: ~11GB")
print(f"  Pruned: ~3.85GB (65% reduction)")
print(f"  Space saved: ~7.15GB")

## 6. Phase 2B: AWQ 4-bit Quantization

**Goal:** Compress weights to 4-bit  
**Input:** 3.85GB pruned model  
**Output:** 1.0GB quantized model  
**Time:** 2-3 hours  

---

### How it works:
- Activation-aware weight quantization
- Preserves most important weights at higher precision
- Group-wise quantization (128 groups)
- Minimal quality loss vs 16-bit

In [ ]:
from awq import AutoAWQForCausalLM  # type: ignore

print("=" * 60)
print("? PHASE 2B: AWQ 4-BIT QUANTIZATION")
print("=" * 60)
print("Target: 3.85GB → 1.0GB (4-bit quantization)")
print("Method: Activation-aware Weight Quantization")
print("Duration: ~1-2 hours\n")

# Load pruned model for quantization
model_awq = AutoAWQForCausalLM.from_pretrained(
    "/content/models/phase2a-pruned",
    safetensors=True
)

# Quantization config
quant_config = {
    "zero_point": True,
    "q_group_size": 128,
    "w_bit": 4,
    "version": "GEMM"
}

# Apply quantization
model_awq.quantize(
    tokenizer,
    quant_config=quant_config,
    calib_data=calibration_data,
    n_samples=512,
    max_calib_samples=512,
    max_calib_seq_len=2048
)

# Save quantized model
print("\n💾 Saving quantized model...")
model_awq.save_quantized("/content/models/phase2b-awq")
tokenizer.save_pretrained("/content/models/phase2b-awq")

print("\n" + "=" * 60)
print("✅ QUANTIZATION COMPLETE!")
print("=" * 60)
print(f"Output: /content/models/phase2b-awq")
print(f"Size: ~1.0GB (4-bit quantized)")
print(f"Expected quality: 87-89% GPT-4")
print("\n➡️ Next: Phase 2C (GGUF Export)")

## 7. Phase 2C: GGUF Export & Final Compression

**Goal:** Export to GGUF format with compression  
**Input:** 1.0GB AWQ model  
**Output:** 480MB GGUF model  
**Time:** 1 hour  

---

### How it works:
- Convert to GGUF format (optimized for llama.cpp)
- Apply Q5_K_M quantization
- Zstandard lossless compression
- Final output: 480MB ready for deployment

In [ ]:
print("=" * 60)
print("📦 PHASE 2C: GGUF EXPORT & COMPRESSION")
print("=" * 60)
print("\nTarget: GGUF Q5_K_M (1.0GB → 480MB)")
print("Method: GGUF + Zstd compression")
print("Time: 1 hour")
print("\nStarting export...\n")

# Convert to GGUF
print("1. Converting to GGUF format...")
!/content/llama.cpp/convert.py \
    /content/models/phase2b-awq \
    --outfile /content/models/phase2c-gguf/model-f16.gguf \
    --outtype f16

# Quantize to Q5_K_M
print("\n2. Applying Q5_K_M quantization...")
!/content/llama.cpp/quantize \
    /content/models/phase2c-gguf/model-f16.gguf \
    /content/models/phase2c-gguf/model-Q5_K_M.gguf \
    Q5_K_M

# Compress with Zstandard
print("\n3. Applying Zstandard compression...")
import zstandard as zstd

with open('/content/models/phase2c-gguf/model-Q5_K_M.gguf', 'rb') as f_in:
    with open('/content/models/phase2c-gguf/model-Q5_K_M.gguf.zst', 'wb') as f_out:
        cctx = zstd.ZstdCompressor(level=19)
        cctx.copy_stream(f_in, f_out)

# Check final size
import os
final_size = os.path.getsize('/content/models/phase2c-gguf/model-Q5_K_M.gguf.zst') / 1024**2

print("\n" + "=" * 60)
print("✅ COMPRESSION COMPLETE!")
print("=" * 60)
print(f"Final model: /content/models/phase2c-gguf/model-Q5_K_M.gguf.zst")
print(f"Final size: {final_size:.0f}MB")
print(f"\nCompressionjourney:")
print(f"  Phase 1: 11GB (English-trained)")
print(f"  Phase 2A: 3.85GB (pruned)")
print(f"  Phase 2B: 1.0GB (quantized)")
print(f"  Phase 2C: {final_size:.0f}MB (GGUF compressed)")
print(f"\n📊 Total reduction: {(1 - final_size/11000) * 100:.1f}%")
print(f"Expected quality: 87-89% GPT-4")

## 8. Test Compressed Model

Quick sanity check to ensure model works correctly.

In [ ]:
# Test the compressed model
print("🧪 Testing compressed model...\n")

# Decompress for testing
with open('/content/models/phase2c-gguf/model-Q5_K_M.gguf.zst', 'rb') as f_in:
    with open('/content/test-model.gguf', 'wb') as f_out:
        dctx = zstd.ZstdDecompressor()
        dctx.copy_stream(f_in, f_out)

# Run simple test with llama.cpp
test_prompt = "Write a Python function to calculate factorial."

print(f"Test prompt: {test_prompt}\n")
print("Response:")
!/content/llama.cpp/main \
    -m /content/test-model.gguf \
    -p "{test_prompt}" \
    -n 128 \
    --temp 0.7 \
    --top-p 0.9

print("\n✅ Model test complete!")

## 9. Download Compressed Model

Download the final 480MB model to your local machine.

In [ ]:
from google.colab import files  # type: ignore

print("📥 Preparing model for download...")
print(f"Size: ~{final_size:.0f}MB")
print("\nClick download when ready...\n")

files.download('/content/models/phase2c-gguf/model-Q5_K_M.gguf.zst')

print("\nSave as: cogumi-llm-480mb.gguf.zst")

## 10. Optional: Upload to HuggingFace

Upload the compressed model to HuggingFace for easy access.

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

# Create repository
repo_id = "YOUR_USERNAME/cogumi-llm-480mb"  # Change this

print(f"📤 Uploading to {repo_id}...")

api.create_repo(repo_id=repo_id, private=True, exist_ok=True)

# Upload compressed model
api.upload_file(
    path_or_fileobj="/content/models/phase2c-gguf/model-Q5_K_M.gguf.zst",
    path_in_repo="model-Q5_K_M.gguf.zst",
    repo_id=repo_id,
    repo_type="model"
)

print(f"\n✅ Model uploaded to: https://huggingface.co/{repo_id}")

---

## ✅ Phase 2 Complete!

### Summary

- ✅ **Phase 2A**: Pruned 65% of neurons (11GB → 3.85GB)
- ✅ **Phase 2B**: Quantized to 4-bit (3.85GB → 1.0GB)
- ✅ **Phase 2C**: Exported to GGUF (1.0GB → 480MB)
- ✅ **Final size**: 480MB (~95% compression from original 16GB)
- ✅ **Expected quality**: 87-89% GPT-4

### Next Steps

1. **Benchmark the model** (Phase 3 evaluation)
   - MMLU, HumanEval, GSM8K
   - Verify quality meets targets

2. **Create domain modifiers** (Phase 3a/3b)
   - Coding modifier (~60MB)
   - Math modifier (~40MB)

3. **Build router** (Phase 4)
   - Modifier selection logic
   - Performance optimization

4. **Deploy locally** (Phase 5)
   - Test on MacBook Air M4
   - Optimize for inference speed

---

**Congratulations! You now have a 480MB English-specialized model!** 🎉